# XGBoost Model
Train and evaluate an XGBoost regressor for sales forecasting.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

# Demo features
dates = pd.date_range('2020-01-01','2023-12-31')
np.random.seed(42)
df = pd.DataFrame({'date':dates})
df['y'] = np.random.normal(5000,1500,len(dates)).clip(0)
df['month']=df['date'].dt.month
df['dayofweek']=df['date'].dt.dayofweek
df['sin_month']=np.sin(2*np.pi*df['month']/12)
df['cos_month']=np.cos(2*np.pi*df['month']/12)
for lag in [7,30]:
    df[f'y_lag_{lag}'] = df['y'].shift(lag)
for w in [7,30]:
    df[f'y_roll_{w}'] = df['y'].rolling(w).mean()
df = df.dropna().reset_index(drop=True)

X = df.drop(columns=['y','date']); y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

model = XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=6, subsample=0.9, colsample_bytree=0.9, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)
mae = mean_absolute_error(y_test, pred)
rmse = mean_squared_error(y_test, pred, squared=False)
r2 = r2_score(y_test, pred)
print({'MAE': mae, 'RMSE': rmse, 'R2': r2})
